## Setup and get data

In [1]:
import pandas as pd
import psycopg2 as pg
import os
import sys
import pandas.io.sql as pd_sql
import numpy as np
import sqlalchemy
project_dir = str(os.path.dirname(os.path.abspath('')))
print(project_dir)
sys.path.append(project_dir)
import matplotlib.pyplot as plt
from src.pickle.pickle_util import save_pickle, load_pickle

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
my_random_state = 72

/Users/erik/metis/data_hailing


### Start with pickle I saved in notebook 02_eda

Alternately, could have loaded it from SQL with the following code:
```
engine = sqlalchemy.create_engine('postgresql://localhost:5432/data_hailing')

query = "SELECT * FROM tnp_trips_2019_pandas WHERE DATE(trip_start_timestamp) = '2019-07-17';"
trips_df = pd.read_sql_query(query, engine)
```

In [2]:
file_path = project_dir + '/data/interim/' + 'trips_2019-07-07_from_02_eda.pickle'
# save_pickle(trips_df, file_path)
trips_df = load_pickle(file_path)

In [3]:
trips_df

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total,shared_trip_authorized,trips_pooled,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
0,2019-07-17 11:30:00,2019-07-17 11:45:00,857.0,4.5,1.703108e+10,1.703106e+10,8.0,6.0,10.0,0.0,2.55,12.55,False,1.0,41.907520,-87.626659,41.949221,-87.651970
1,2019-07-17 19:00:00,2019-07-17 19:15:00,1181.0,5.6,1.703132e+10,1.703122e+10,32.0,22.0,12.5,12.0,2.55,27.05,False,1.0,41.884987,-87.620993,41.928465,-87.695087
2,2019-07-17 23:45:00,2019-07-17 23:45:00,779.0,5.4,1.703108e+10,1.703183e+10,8.0,6.0,10.0,0.0,2.55,12.55,False,1.0,41.900221,-87.629105,41.945170,-87.668794
3,2019-07-17 18:15:00,2019-07-17 19:15:00,3877.0,26.5,NaN,1.703132e+10,NaN,32.0,42.5,0.0,2.55,45.05,False,1.0,NaN,NaN,41.884987,-87.620993
4,2019-07-17 22:30:00,2019-07-17 22:45:00,671.0,4.2,1.703108e+10,1.703106e+10,8.0,6.0,7.5,0.0,2.55,10.05,False,1.0,41.892042,-87.631864,41.943155,-87.640698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287209,2019-07-17 22:15:00,2019-07-17 22:15:00,407.0,1.3,1.703108e+10,1.703108e+10,8.0,8.0,5.0,1.0,2.55,8.55,False,1.0,41.893216,-87.637844,41.907492,-87.635760
287210,2019-07-17 23:30:00,2019-07-17 23:45:00,712.0,5.7,1.703184e+10,1.703184e+10,24.0,31.0,10.0,2.0,2.55,14.55,False,1.0,41.898306,-87.653614,41.855501,-87.683227
287211,2019-07-17 20:00:00,2019-07-17 20:15:00,1254.0,7.3,1.703108e+10,1.703116e+10,8.0,16.0,12.5,5.0,2.55,20.05,False,1.0,41.892073,-87.628874,41.950212,-87.712814
287212,2019-07-17 23:45:00,2019-07-18 00:00:00,827.0,5.2,NaN,NaN,35.0,61.0,10.0,0.0,2.55,12.55,False,1.0,41.835118,-87.618678,41.809018,-87.659167


### Get income data from SQL

In [4]:
engine = sqlalchemy.create_engine('postgresql://localhost:5432/data_hailing')
query = "SELECT * FROM per_capita_income_chicago_tidy limit 500;"
income_df = pd.read_sql_query(query, engine).dropna()

## Merge data on community area number

In [5]:


income_df = income_df[['community_area_number', 'per_capita_income_']]

trips_df = trips_df.loc[trips_df.fare > 0.001]
trips_df = trips_df.dropna()
trips_df['community_area_number'] = trips_df['pickup_community_area'].astype(int)

trips_df = trips_df[['community_area_number', 'trip_start_timestamp', 'trip_seconds', 'trip_miles','fare', 'pickup_centroid_latitude','pickup_centroid_longitude']]

df_merge_trip_income = pd.merge(trips_df, income_df, on='community_area_number')

In [6]:
df_merge_trip_income

,community_area_number,trip_start_timestamp,trip_seconds,trip_miles,fare,pickup_centroid_latitude,pickup_centroid_longitude,per_capita_income_
0,8,2019-07-17 11:30:00,857.0,4.5,10.0,41.907520,-87.626659,88669
1,8,2019-07-17 23:45:00,779.0,5.4,10.0,41.900221,-87.629105,88669
2,8,2019-07-17 22:30:00,671.0,4.2,7.5,41.892042,-87.631864,88669
3,8,2019-07-17 16:45:00,460.0,1.0,10.0,41.891972,-87.612945,88669
4,8,2019-07-17 09:30:00,744.0,2.0,7.5,41.905858,-87.630865,88669
...,...,...,...,...,...,...,...,...
185174,54,2019-07-17 14:30:00,864.0,3.9,7.5,41.650222,-87.599463,8201
185175,55,2019-07-17 17:45:00,387.0,1.8,5.0,41.651922,-87.564929,22677
185176,55,2019-07-17 18:00:00,1112.0,8.7,12.5,41.651922,-87.564929,22677
185177,74,2019-07-17 08:30:00,2265.0,13.4,22.5,41.687239,-87.719313,34381


## Engineer Features

### Part of day

In [7]:
df_merge_trip_income.loc[:, 'part_of_day'] = [hour//4 for hour in df_merge_trip_income.trip_start_timestamp.dt.hour]

### Create surge estimate

In [8]:
linreg = LinearRegression()
y = df_merge_trip_income.loc[:, 'fare']
X = df_merge_trip_income.loc[:, ['trip_miles', 'trip_seconds']]

linreg.fit(X, y)

pred_fare = linreg.predict(X)
actual_fare = df_merge_trip_income.fare
surge_estimate = actual_fare / pred_fare
df_merge_trip_income['surge_estimate'] = surge_estimate
surge_estimate.describe()

count    185179.000000
mean          1.010462
std           0.306218
min           0.107673
25%           0.881222
50%           1.000192
75%           1.103980
max          12.975854
Name: fare, dtype: float64

## Setup get X and y

In [9]:
def get_X(df_merge_trip_income, dummies_on_part_of_day=True):
    X = df_merge_trip_income[['pickup_centroid_latitude', 'pickup_centroid_longitude', 'per_capita_income_', 'part_of_day']]
    if dummies_on_part_of_day:
        X = pd.get_dummies(X, columns=['part_of_day'])
    else:
        X = X.drop['part_of_day']
    return X


def get_y(df_merge_trip_income, surge_cutoff=1.1):
    y = df_merge_trip_income.loc[:, 'surge_estimate'] > surge_cutoff
    return y

In [10]:
X = get_X(df_merge_trip_income)
y = get_y(df_merge_trip_income)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=my_random_state)

## Setup a function for custom cross validation on random forest

I selected random forest because it did the best on precision.

In [11]:
def run_cv_random_forest(X, y, n_estimators=10, predict_proba_threshold=.5):
    #run the CV
    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    kf = KFold(n_splits=5, shuffle=True, random_state = my_random_state)

    accuracy_scores = []
    precision_scores = []
    recall_scores = []


    for try_ind, val_ind in kf.split(X_train,y_train):

        X_try, y_try = X[try_ind], y[try_ind]
        X_val, y_val = X[val_ind], y[val_ind] 

        rf = RandomForestClassifier(n_estimators=n_estimators, random_state=my_random_state)
        rf.fit(X_try, y_try);

#         y_pred = rf.predict(X_val)
        y_pred = (rf.predict_proba(X_val))[:,1] > predict_proba_threshold
        accuracy_scores.append(accuracy_score(y_val, y_pred))
        precision_scores.append(precision_score(y_val, y_pred))
        recall_scores.append(recall_score(y_val, y_pred))


    print('accuracy_scores: ', accuracy_scores)
    print(f'accuracy_scores: {np.mean(accuracy_scores):.4f} +- {np.std(accuracy_scores):.4f}')

    print('precision_scores: ', precision_scores)
    print(f'precision_scores: {np.mean(precision_scores):.3f} +- {np.std(precision_scores):.3f}')

    print('recall_scores: ', recall_scores)
    print(f'recall_scores: {np.mean(recall_scores):.5f} +- {np.std(recall_scores):.5f}')

    return rf

In [12]:
run_cv_random_forest(X, y, n_estimators=40, predict_proba_threshold=0.45);

accuracy_scores:  [0.7259603268891529, 0.7181121071389999, 0.7283723944270439, 0.7156640385930806, 0.7193260368663594]
accuracy_scores: 0.7215 +- 0.0048
precision_scores:  [0.35, 0.23178807947019867, 0.29213483146067415, 0.3, 0.3108108108108108]
precision_scores: 0.297 +- 0.038
recall_scores:  [0.0036900369003690036, 0.004516711833785004, 0.0034629728289824187, 0.003434240651233783, 0.005943152454780362]
recall_scores: 0.00421 +- 0.00095


## Run this model on 'holdout' test set of data

In [13]:
def random_forest_test_set(n_estimators=40, predict_proba_threshold=.45):
    X = get_X(df_merge_trip_income)
    y = get_y(df_merge_trip_income)
    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=my_random_state)
    
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=my_random_state)
    rf.fit(X_train, y_train);
    
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
#         y_pred = rf.predict(X_val)
    y_pred = (rf.predict_proba(X_test))[:,1] > predict_proba_threshold
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred))


    print('accuracy_score: ', accuracy_scores[0])

    print('precision_score: ', precision_scores[0])

    print('recall_score: ', recall_scores[0])

    return rf

In [14]:
rf = random_forest_test_set();

accuracy_score:  0.7375094502646075
precision_score:  0.2651356993736952
recall_score:  0.010648109331768256


In [15]:
rf.feature_importances_

array([0.29218812, 0.31202335, 0.14484304, 0.01142502, 0.01571451,
       0.02483924, 0.0262007 , 0.1447239 , 0.02804212])

In [16]:
X.columns

Index(['pickup_centroid_latitude', 'pickup_centroid_longitude',
       'per_capita_income_', 'part_of_day_0', 'part_of_day_1', 'part_of_day_2',
       'part_of_day_3', 'part_of_day_4', 'part_of_day_5'],
      dtype='object')

The important columns are: 
```
'pickup_centroid_latitude': 0.292, 
'pickup_centroid_longitude': 0.312,
'per_capita_income_': 0.145,
'part_of_day_4': 0.145
```
The rest are less than three percent important
   

## Pickle off model for future use

In [18]:
file_path = project_dir + '/models/' + 'random_forest_from_02_feat_eng_and_model_notebook.pickle'
save_pickle(rf, file_path)

# Also, save off this data as a csv for use in visualization

In [22]:
file_path = project_dir + '/data/processed/' + 'df_merge_trip_income_from_03_feat_notebook'
assert not os.path.exists(file_path)
df_merge_trip_income.to_csv(file_path, index=False)